In [119]:
from PIL import Image
import pytesseract
import requests
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [120]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [121]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return text

In [122]:
import telebot
API_TOKEN = '1057517582:AAHSGWUKn-4JtVGdGbMjFwHBKTYWf6I7EsY'
bot = telebot.TeleBot(API_TOKEN)


@bot.message_handler(content_types=['photo', 'document', 'text'])
def get_text_messages(message):
    if message.content_type == 'photo':
        bot.send_message(
            message.from_user.id,
            "Твое изображение получил, сейчас обработаю и скину текст. Местами могу ошибаться, я еще только учусь."
        )
        a = message.json
        file_id = a['photo'][-1]['file_id']
        file_info = bot.get_file(file_id)

        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
            API_TOKEN, file_info.file_path))

        with open('C:/Users/Fedor/Documents/Edu/ComputerNetworks/file.jpg',
                  'wb') as f:
            f.write(file.content)

        bot.send_message(message.from_user.id, "Так, картинка уже у меня...")
        file = Image.open(
            'C:/Users/Fedor/Documents/Edu/ComputerNetworks/file.jpg')
        im = file.convert('L')
        text = pytesseract.image_to_string(im, lang='rus')
        print(text)
        bot.send_message(message.from_user.id, text)

    if message.content_type == 'document':
        bot.send_message(message.from_user.id,
                         "Твой pdf получил, сейчас обработаю...")
        a = message.json
        file_id = a['document']['file_id']
        file_info = bot.get_file(file_id)
        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
            API_TOKEN, file_info.file_path))
        path = 'C:/Users/Fedor/Documents/Edu/ComputerNetworks/' + str(
            a['document']['file_name'])
        with open(path, 'wb') as f:
            f.write(file.content)

        bot.send_message(message.from_user.id, "Так, уже скачал...")
        text = extract_text_from_pdf(path)
        bot.send_message(message.from_user.id, text)
        print(text)

    if message.content_type == 'text':
        if message.text == "/command1":
            bot.send_message(
                message.from_user.id,
                'Привет еще раз! Просто отправь мне файл формата PDF или обычный скриншот, я постараюсь вытащить оттуда текст :)'
            )
        if message.text == "/command2":
            bot.send_message(
                message.from_user.id,
                "Я самый обычный телеграмм-бот, использую технологии компьютерного зрения для обработки фотографий. Мой код лежт тут: *ссылка на гит*"
            )
        if message.text == "/start":
            bot.send_message(
                message.from_user.id,
                "Привет, {0}! Меня зовут TextGrabber. Я помогу тебе достать текст с картинки или PDF файла. Нажми /command1"
                .format(str(message.from_user.first_name)))


bot.polling(none_stop=True, interval=0)